**Installing Dependencies**

In [ ]:
!pip install tweepy

In [ ]:
!pip install textblob

**Importing Libraries**

In [ ]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
from textblob import TextBlob


**Basic Analysis**

In [ ]:
class TwitterClient(object):
    def __init__(self):
        # keys and tokens from the Twitter Dev Console 
        # bearer token: 
        #AAAAAAAAAAAAAAAAAAAAACB%2BTQEAAAAAjs7TGTapyQgMvjy7NtGPSq0AU3s%3DH5sCcmkM1HHqRENtwD8mpJhfT6c53jfsSj6Lmu292fEFGmcNyT
        consumer_key = 'iMeyQV9aWLo55j3IMpKhNQmO4'
        consumer_secret = 'y9cGxmiwNlAVZEtKh2DuTSGAWqK2Qc7JktC1d5bwsq2AJjssbk'
        access_token = '1076128111345823745-nDhq4mnCCNJfHhhpQhapyuNaJbpiKP'
        access_token_secret = 'EPOHM8TFUhmpNgUcVQ4T9I9ZIweUFzLHUsXNkxNtK2JRH'
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [ ]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'PM Kisan', count = 2000)
  
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \ ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
    data = {'ptweets': ptweets, 'ntweets': ntweets}
    df = pd.DataFrame(ptweets)
    ef = pd.DataFrame(tweets)
    cf = pd.DataFrame(ntweets)
    df.to_csv('ptweets.csv')
    ef.to_csv('tweets.csv')
    cf.to_csv('ntweets.csv')
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
  
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
  
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 20.338983050847457 %
Negative tweets percentage: 8.474576271186441 %
Neutral tweets percentage: 71.1864406779661 % \ 


Positive tweets:
Graphs clearly demonstrating #PM_KUSUM ( Pradhan Mantri Kisan Urja Suraksha Evam Utthaan Mahabhiyan ) as an unique… https://t.co/pePR2N48lV
RT @ISNTiwari: ⚫ दरअसल सरकार को यह सारा डेटा उठाकर देशी विदेशी कम्पनियों के हाथों करना है, ये सब नयी विश्व व्यवस्था new world order की तैया…
RT @HitenPithadiya: 21000 crores Heroin seized from private port of PM Modi's friend Gautam Adani. 
Is this the real anti-national activiti…
RT @dhrubachoudhur5: The term ‘Nehruvian’ has come to symbolise all the great qualities that India’s first PM Nehru, was known for in his l…
RT @DDKisanChannel: Watch Krishi Darshan, DD Kisan’s special program on the use of Advanced Technology in Agriculture sector. Today at 5:30…
Watch Krishi Darshan, DD Kisan’s special program on the use of Advanced Technology in Agriculture sector. Today at… https://t.co/B2